In [1]:
#imports
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import tensorflow as tf
import numpy as np
!pip install tflearn
import tflearn
import random
import pickle
from time import*
import json

#imports git files
import pandas as pd

!rm -r MHBot #Use this line to delete the git repo file before reloading with line below to ensure updated files
!git clone https://github.com/Banani456/MHBot.git

with open('/content/MHBot/intents.json') as json_data:
    intents = json.load(json_data)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 KB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=fdd6ebf5f7baa0a0b0035ed767a97ab50210cbb020b92074024175f7fd62f4ad
  Stored in directory: /root/.cache/pip/wheels/65/9b/15/cb1e6b279c14ed897530d15cfd7da8e3df8a947e593f5cfe59
Successfully built tflearn


Instructions for updating:
non-resource variables are not supported in the long term


rm: cannot remove 'MHBot': No such file or directory
Cloning into 'MHBot'...
remote: Enumerating objects: 179, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 179 (delta 46), reused 0 (delta 0), pack-reused 94
Receiving objects: 100% (179/179), 9.59 MiB | 19.96 MiB/s, done.
Resolving deltas: 100% (87/87), done.


{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Sup'],
   'responses': ['Hello, how are you?',
    "Good to see you, what's up?",
    'Hi there, how can I help?',
    "Heyy! How's it going?",
    "Hiya, how're ya doin'?"],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'anger',
   'patterns': ['i am just feeling cranky and blue',
    'i feel bitchy but not defeated yet',
    'i know what it feels like he stressed glaring down at her as she squeezed more soap onto her sponge',
    'i feel like taking a whack at someone s eye and spitting on it a cranky old lady i try to cheer myself up',
    'i

In [2]:
########### READING FILES ########### intents will train our model
#empty arrays to store data
words = []
classes = []
documents = []
ignore = [] #will ignore these characters for now

# loop through each sentence in the intent's patterns
# emotions list to ananlyze (sadness, anger, love, surprise, fear, joy)
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        # add word to the words list
        words.extend(w)
        # add word(s) to documents
        documents.append((w, intent['tag']))
        # add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


############################################
# stemming, lowercasing and removing duplicate words for efficiency
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)



214 documents
28 classes ['addictive', 'anger', 'anxiety', 'compulsive', 'conduct', 'disorderquestion', 'dissociative', 'eating', 'elimination', 'fear', 'goodbye', 'greeting', 'joy', 'love', 'mood', 'neurocognitive', 'neurodevelopmental', 'paraphilias', 'personality', 'psychotic', 'sadness', 'selfharm', 'sexual', 'sleepwake', 'somatic', 'surprise', 'thanks', 'trauma']
865 unique stemmed words ["'re", "'s", "'ve", '(', ')', ',', '.', '?', '[', ']', 'a', 'abdomin', 'abl', 'abnorm', 'about', 'abs', 'absolv', 'abus', 'achiev', 'act', 'adequ', 'adult', 'adv', 'affect', 'afford', 'against', 'aggress', 'agit', 'al', 'align', 'allow', 'almost', 'alon', 'also', 'alt', 'altern', 'alway', 'am', 'amount', 'an', 'and', 'and/or', 'angry', 'angry/irritable', 'anim', 'annoy', 'antivir', 'anxy', 'any', 'anyon', 'anyth', 'appear', 'appetit', 'apprecy', 'ar', 'are', 'argu', 'argumentative/defiant', 'army', 'around', 'as', 'asham', 'asleep', 'aspect', 'assocy', 'at', 'attack', 'attitud', 'auth', 'avast', 

In [3]:
#training data list
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming the words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

#shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

#training lists
train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-3-f5a8217005a4>:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [4]:
###########################################BUILDING NEURAL NETWORK FROM SCRATCH#####################################################
tf.compat.v1.reset_default_graph()

# Building neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

# Start training
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )
     
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

with open('/content/MHBot/intents.json') as json_data:
    intents = json.load(json_data)

Training Step: 26999  | total loss: 0.00018 | time: 0.171s
| Adam | epoch: 1000 | loss: 0.00018 - acc: 1.0000 -- iter: 208/214
Training Step: 27000  | total loss: 0.00017 | time: 0.176s
| Adam | epoch: 1000 | loss: 0.00017 - acc: 1.0000 -- iter: 214/214
--


In [5]:
# load the saved model
model.load('./model.tflearn')

def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bag_of_words(sentence, words, show_details=False):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

##

#Adding some context to the conversation
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bag_of_words(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def find_tag(input):
    classification = classify(input)
    # if we have a classification then find the matching intent tag
    if classification:
          for i in intents['intents']:
              # find a tag matching the first result
              if i['tag'] == classification[0][0]:
                return(i['tag'])
                    

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [12]:
#actual bot code
r = input("Me: ")
while(find_tag(r) != "goodbye"):
    sleep(1)
    print("Mia: ", end = '')
    response(r)
    r = input("Me: ")

sleep(1)
print("Mia: ", end = '')
response(r)


Me: hey mia!
Mia: Good to see you, what's up?
Me: im a little not okay i wish my friend would text me back
Mia: This seems to sadden you
Me: yeah i really am, theyre not doing well at the moment but its making me sad and i cant seem to do daily tasks well because of how worried i am
Mia: Exciting!
Me: ...huh
Mia: Hello, how are you?
Me: um okay then mia, im still RIGHT HErE, uh and clearly i kinda want to cry
Mia: Awww
Me: -_-
Mia: Hiya, how're ya doin'?
Me: *sigh*
Mia: Heyy! How's it going?
Me: i think i might have a disorder or mental health issue, can you help me identify which one
Mia: Yes, if you give me some symptoms, I could try estimating what type of disorders you could have
Me: uh okay, i dont have any motivation, im not eating well, sleeping on time or waking up on time, i am very distracted, moody and honestly really really sad and empty all the time
Mia: I think you have sleep-wake disorders - refer to this site for more information https://www.mayoclinic.org/diseases-cond